In [24]:
!pip install captcha

In [25]:
from captcha.image import ImageCaptcha
import os
import random
import string

def generate_captcha_images(output_folder, num_images=10000):
    os.makedirs(output_folder, exist_ok=True)
    chars = string.ascii_letters + string.digits
    image = ImageCaptcha(width=160, height=60)

    for i in range(num_images):
        text = ''.join(random.choices(chars, k=4))
        image.write(text, os.path.join(output_folder, f"{text}.png"))

generate_captcha_images("data/captchas", num_images=10000)

In [26]:
import cv2
import numpy as np

def split_captcha(image_path, output_folder):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    os.makedirs(output_folder, exist_ok=True)
    regions = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > 10 and w > 10:
            regions.append((x, y, w, h))

    regions = sorted(regions, key=lambda r: r[0])

    for idx, (x, y, w, h) in enumerate(regions):
        letter = img[y:y+h, x:x+w]
        letter = cv2.resize(letter, (28, 28))
        letter_path = os.path.join(output_folder, f"letter_{idx}.png")
        cv2.imwrite(letter_path, letter)

for captcha_file in os.listdir("data/captchas"):
    split_captcha(f"data/captchas/{captcha_file}", "data/letters")


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(62, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 62)                  │           7,998 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 231,742 (905.24 KB)

 Trainable params: 231,742 (905.24 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def load_data(folder):
    data, labels = [], []
    chars = string.ascii_letters + string.digits

    for file in os.listdir(folder):
        label = file.split("_")[1][0]
        if label in chars:
            img = load_img(os.path.join(folder, file), color_mode="grayscale")
            img = img_to_array(img) / 255.0
            data.append(img)
            labels.append(chars.index(label))

    return np.array(data), to_categorical(labels, num_classes=62)

train_data, train_labels = load_data("data/letters")


In [29]:
history = model.fit(train_data, train_labels, epochs=20, batch_size=64, validation_split=0.2)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 4.1050 - val_accuracy: 0.0000e+00 - val_loss: 4.2814
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 3.8816 - val_accuracy: 0.0000e+00 - val_loss: 4.3602
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 3.6745 - val_accuracy: 0.0000e+00 - val_loss: 4.4639
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 3.4126 - val_accuracy: 0.0000e+00 - val_loss: 4.6177
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 3.0844 - val_accuracy: 0.0000e+00 - val_loss: 4.8318
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 2.6749 - val_accuracy: 0.0000e+00 - val_loss: 5.1365
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 2.1805 - val_accuracy: 0.0000e+00 - val_loss: 5.5891
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 1.0000 - loss: 1.6169 - val_accur

In [30]:
generate_captcha_images("sample_catptcha_images", num_images=2)

In [31]:
def predict_captcha(image_path):
    split_captcha(image_path, "temp_letters")
    prediction = []

    for letter_file in sorted(os.listdir("temp_letters")):
        letter_img = load_img(f"temp_letters/{letter_file}", color_mode="grayscale")
        letter_img = img_to_array(letter_img) / 255.0
        letter_img = np.expand_dims(letter_img, axis=0)
        prediction.append(model.predict(letter_img).argmax())

    chars = string.ascii_letters + string.digits
    return ''.join([chars[p] for p in prediction])

captcha_text = predict_captcha("sample_captcha_images/captcha1.png")
print(f"Predicted CAPTCHA: {captcha_text}")


Predicted CAPTCHA: 
